In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, merge, add
from keras.applications import MobileNet, VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
test_id = []
train_id = []

In [3]:
files = os.listdir('images/train/1')
print(len(files))
for index in range(len(files)):
  train_id.append(str(files[index]).split('.')[0])
files = os.listdir('images/train/0')
for index in range(len(files)):
  train_id.append(str(files[index]).split('.')[0])

10463


In [4]:
files = os.listdir('images/test/1')
print(len(files))
for index in range(len(files)):
  test_id.append(str(files[index]).split('.')[0])
files = os.listdir('images/test/0')
for index in range(len(files)):
  test_id.append(str(files[index]).split('.')[0])

2407


In [5]:
final = pd.read_csv('final_merged.csv')

In [30]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()

train_df = final[final['tweet_id'].isin(train_id)]
test_df = final[final['tweet_id'].isin(test_id)]

In [31]:

traininputImages = []
testinputImages = []
# loop over the input house paths
wrong_id = []
for index,rows in train_df.iterrows():
    try:
        if(rows['annotation']==0):
            #print('images/train/0/'+str(rows['tweet_id'])+'.jpg')
            train_image = cv2.imread('images/train/0/'+str(rows['tweet_id'])+'.jpg')
            #print(train_image.size)
            train_image = cv2.resize(train_image, (224, 224))
        else:
            train_image = cv2.imread('images/train/1/'+str(rows['tweet_id'])+'.jpg')
            train_image = cv2.resize(train_image, (224, 224))
        traininputImages.append(train_image)
    except Exception as e:
        print(rows['tweet_id'])
        print(e)
        wrong_id.append(rows['tweet_id'])

for index,rows in test_df.iterrows():
    try:
        if(rows['annotation']==0):
            test_image = cv2.imread('images/test/0/'+str(rows['tweet_id'])+'.jpg')
            test_image = cv2.resize(test_image, (224, 224))
        else:
            test_image = cv2.imread('images/test/1/'+str(rows['tweet_id'])+'.jpg')
            test_image = cv2.resize(test_image, (224, 224))
        testinputImages.append(test_image)
    except:
        print(rows['tweet_id'])
        wrong_id.append(rows['tweet_id'])

traininputImages = np.array(traininputImages)
testinputImages = np.array(testinputImages)

325150935364022272
OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

325177553713238016
OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

325157607427948544
OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

325164356943900672
OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

325154445354758144
OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

325306272842911744
OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion

In [32]:
train_df = train_df[~train_df['tweet_id'].isin(wrong_id)]
test_df = test_df[~test_df['tweet_id'].isin(wrong_id)]

In [33]:
y_train = train_df['annotation']
X_train = train_df['tweets']
y_test = test_df['annotation']
X_test = test_df['tweets']

In [34]:
y_train.shape

(17717,)

In [35]:
train_texts = [s.lower() for s in X_train]
test_texts = [s.lower() for s in X_test]

In [36]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.layers import Bidirectional, GlobalMaxPool1D, Bidirectional, LSTM, GRU
from keras.models import Model

In [37]:
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)
# If we already have a character list, then replace the tk.word_index
# If not, just skip below part

In [38]:
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

In [39]:
# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

In [40]:
from keras.utils import to_categorical

train_classes = to_categorical(y_train)
test_classes = to_categorical(y_test)

In [41]:
vocab_size = len(tk.word_index)
print(vocab_size)

69


In [42]:
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size))

for char,i in tk.word_index.items():
  onehot = np.zeros(vocab_size)
  onehot[i-1] = 1
  embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)
print(embedding_weights.shape)

(70, 69)


In [43]:
embedding_size = 69
input_size = 1014
conv_layers = [[256, 7, 3],[256, 7, 3],[256, 3, -1],[256, 3, -1],[256, 3, -1],[256, 3, 3]]
fully_connected_layers = [1024, 1024]
num_of_classes = 2
dropout_p=0.5
optimizer = 'adam'
loss = 'binary_crossentropy'

In [44]:
embedding_layer = Embedding(vocab_size+1, embedding_size, input_length=input_size, weights = [embedding_weights])


In [45]:
text_inputs = Input(shape = (input_size,), name = 'input', dtype = 'int64')

x = embedding_layer(text_inputs)

for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size!=-1:
        x = MaxPooling1D(pool_size = pooling_size)(x)
x = Bidirectional(GRU(60, return_sequences=True,name='lstm_layer',dropout=0.2,recurrent_dropout=0.2))(x)
x = GlobalMaxPool1D()(x)

for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation = 'relu')(x)
    x = Dropout(dropout_p)(x)

# predictions = Dense(num_of_classes, activation = 'softmax')(x)
# text_model = Model(inputs = inputs, outputs = predictions)
# text_model.compile(optimizer = optimizer, loss=loss, metrics = ['accuracy'])
# text_model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [46]:
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train_text = train_data[indices]
y_train_text = train_classes[indices]

x_test_text = test_data
y_test_text = test_classes

In [47]:
base_model=VGG16(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

for layer in base_model.layers:
    layer.trainable = False
    
i_x=base_model.output
i_x=GlobalAveragePooling2D()(i_x)
i_x=Dense(1024,activation='relu')(i_x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# i_x=Dense(1024,activation='relu')(i_x) #dense layer 2
# i_x=Dense(512,activation='relu')(i_x) #dense layer 3
# preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [48]:
joint = add([x, i_x])
joint = Dense(512, activation='relu')(joint)
joint = Dropout(0.5)(joint)

In [49]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import os.path

In [50]:
len(testinputImages)

4191

In [51]:
test_df.shape

(4191, 7)

In [52]:
predictions = Dense(2, activation='sigmoid')(joint)

In [53]:
full_model = Model(inputs=[base_model.input, text_inputs], outputs=[predictions])

full_model.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

full_model.summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 1014)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1014, 69)     4830        input[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1008, 256)    123904      embedding_1[0][0]                
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None, None, 3 0                                          

In [ ]:
history = full_model.fit([traininputImages, train_data], train_classes,
                         epochs=10, batch_size=128,
                         verbose=1, validation_split=0.2, shuffle=True,
                        validation_data=([testinputImages,test_data], test_classes))